In [1]:
import os

In [79]:
import fitsio
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy import stats

import coord
import galsim
from sklearn.neighbors import BallTree

In [3]:
import lib

In [52]:
import os

In [53]:
os.environ["IMSIM_DATA"] = "/global/cfs/cdirs/desbalro"

In [61]:
SHEAR_CATALOGS = {
    shear_step: os.path.join(
        "/pscratch/sd/s/smau/y6-image-sims-cats-noshear/fiducial",
        shear_step,
        "metadetect_cutsv6_all.h5",
    )
    for shear_step in lib.const.SHEAR_STEPS
}

In [58]:
MATCH_CATALOGS = {
    shear_step: os.path.join(
        "/pscratch/sd/s/smau/fiducial-matches-noshear",
        f"match_{shear_step}.hdf5",
    )
    for shear_step in lib.const.SHEAR_STEPS
}

In [4]:
for shear_step in lib.const.SHEAR_STEPS:
    with (
        h5py.File(
            lib.const.SIM_MATCH_CATALOGS[shear_step],
            mode="r",
        ) as hf_wide,    
        h5py.File(
            MATCH_CATALOGS[shear_step],
            mode="r",
        ) as hf_match,
    ):
        wide_uid = hf_wide["mdet"]["noshear"]["uid"][:]
        match_uid = hf_match["mdet"]["noshear"]["uid"][:]

        wide_z = hf_wide["mdet"]["noshear"]["z"][:]
        match_z = hf_match["mdet"]["noshear"]["z"][:]

        try:
            np.testing.assert_equal(wide_z, match_z)
        except:
            print(shear_step)

        # print(
        #     shear_step,
        #     sum(~np.isnan(wide_uid)),
        #     sum(~np.isnan(match_uid)),
        #     100 * sum(~np.isnan(match_uid)) / sum(~np.isnan(wide_uid)),
        # )

g1_slice=-0.02__g2_slice=0.00__g1_other=0.00__g2_other=0.00__zlow=0.0__zhigh=6.0
g1_slice=0.02__g2_slice=0.00__g1_other=-0.02__g2_other=0.00__zlow=2.4__zhigh=2.7


In [5]:
shear_step = "g1_slice=-0.02__g2_slice=0.00__g1_other=0.00__g2_other=0.00__zlow=0.0__zhigh=6.0"

In [62]:
shear = h5py.File(SHEAR_CATALOGS[shear_step])

In [63]:
match = h5py.File(MATCH_CATALOGS[shear_step])

In [64]:
tilenames = np.unique(shear["mdet"]["noshear"]["tilename"][:])

In [67]:
# find the tile where we broke on...

In [66]:
for tilename in tilenames:
    in_tile = (shear["mdet"]["noshear"]["tilename"][:] == tilename)
    match_in_tile = match["mdet"]["noshear"]["uid"][in_tile]
    if np.all(np.isnan(match_in_tile)):
        print(tilename)
        break

b'DES0402-6039'


In [68]:
for mdet_step in lib.const.MDET_STEPS:
    print(
        tilename,
        mdet_step,
        sum(shear["mdet"][mdet_step]["tilename"][:] == tilename),
    )

b'DES0402-6039' noshear 1
b'DES0402-6039' 1p 1
b'DES0402-6039' 1m 1
b'DES0402-6039' 2p 0
b'DES0402-6039' 2m 1


In [69]:
# so this particular tile has a _single_ object detected in it in the noshear catalog, and
# happens to have no objects in the 2p catalog... that's neat

In [70]:
# let's look at `instance.py` internals, now

In [76]:
shear_args = lib.util.parse_shear_arguments(shear_step)

In [78]:
g1_slice = shear_args["g1_slice"]
g2_slice = shear_args["g2_slice"]
g1_other = shear_args["g1_other"]
g2_other = shear_args["g2_other"]
zlow = shear_args["zlow"]
zhigh = shear_args["zhigh"]

In [71]:
hf_imsim = shear

In [72]:
wcs = lib.util.load_wcs(tilename)

In [73]:
truth_table = fitsio.FITS(lib.const.TRUTH_CATALOGS[tilename.astype(str)])[1]

In [80]:
shear_slice = galsim.Shear(g1=g1_slice, g2=g2_slice)
shear_other = galsim.Shear(g1=g1_other, g2=g2_other)

sheared_radec = []
for i in range(truth_table.get_nrows()):
    ra = truth_table["ra_sim"][i]
    dec = truth_table["dec_sim"][i]
    redshift = truth_table["photoz"][i]

    world_pos = coord.CelestialCoord(ra=ra * coord.degrees, dec=dec * coord.degrees)
    u, v = wcs.center.project(world_pos, projection="gnomonic")
    pos = galsim.PositionD(u.rad, v.rad)

    if (redshift > zlow) & (redshift <= zhigh):
        sheared_pos = pos.shear(shear_slice)
    else:
        sheared_pos = pos.shear(shear_other)

    u2 = sheared_pos.x * coord.radians
    v2 = sheared_pos.y * coord.radians
    sheared_world_pos = wcs.center.deproject(u2, v2, projection="gnomonic")

    sheared_ra = sheared_world_pos.ra.deg
    sheared_dec = sheared_world_pos.dec.deg

    sheared_radec.append((sheared_ra, sheared_dec))

truth_points = np.deg2rad(np.array(sheared_radec))


In [83]:
bt = BallTree(
    truth_points,
    metric="haversine",
)
query_radius_arcsec = 1
query_radius = np.deg2rad(query_radius_arcsec / 60 / 60)

In [84]:
for mdet_step in lib.const.MDET_STEPS:
    
    in_tile = hf_imsim["mdet"][mdet_step]["tilename"][:] == tilename
    
    catalog_indices = np.indices(
        in_tile.shape,
    ).ravel()
    
    
    observed_table = {
        key: hf_imsim["mdet"][mdet_step][key][in_tile]
        for key in [
            "ra_noshear", "dec_noshear",
            "pgauss_band_flux_g", "pgauss_band_flux_err_g",
            "pgauss_band_flux_r", "pgauss_band_flux_err_r",
            "pgauss_band_flux_i", "pgauss_band_flux_err_i",
            "pgauss_band_flux_z", "pgauss_band_flux_err_z",
        ]
    }
    
    observed_points = np.deg2rad(
        np.array([
            observed_table["ra_noshear"],
            observed_table["dec_noshear"],
        ]).T
    )

    indices = bt.query_radius(
        observed_points,
        r=query_radius,
    )

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

In [86]:
len(observed_points)

0